In [ ]:
import datetime
from multiprocessing import Pool

import numpy as np
import pandas as pd

from test.test_baa import execute_experiment

In [ ]:
use_parallel = True

In [ ]:
num_qubits = 7
entanglement_bounds = (0.4, 1.0)
max_fidelity_loss = np.linspace(0.1, 1.0, 10)

data = [(i, num_qubits, entanglement_bounds, max_fidelity_loss) for i in range(100)]
if use_parallel:
    with Pool() as pool:
        result = pool.starmap(execute_experiment, data)
else:
    result = [execute_experiment(*d) for d in data]

In [ ]:
df = pd.concat(result, ignore_index=True)
df

In [ ]:
# Tests:
# Calculation Tests
# The benchmark will not change the state at all, so it must be essentially zero
df['benchmark_fidelity_loss_pass'] = np.abs(df['real_fidelity_loss_benchmark']) < 1e-6
# The expected / predicted fidelity loss must be less or equal to the max fidelity loss
df['approximation_calculation_pass'] = df['max_fidelity_loss'] >= df['total_fidelity_loss']

# The real Tests
# The real measured fidelity measure must be less or equal to the configured mx fidelity loss
df['real_approximation_calculation_pass'] = df['real_fidelity_loss'] < df['max_fidelity_loss']
# The predicted maximum CNOT gates and the no-approximation count should be within 10%
df['cnot_prediction_calculation_pass'] = np.abs(df['real_cnots_no_approx'] - df['cnots']) < 0.1 * df['cnots']
# The predicted CNOT gates should be within an error margin of 10%
df['saved_cnots_calculation_pass'] = np.abs(df['cnots'] - df['total_saved_cnots'] - df['real_cnots']) <= 0.1 * (df['cnots'] - df['total_saved_cnots'])

In [ ]:
timestamp_sec = int(datetime.datetime.now().timestamp())
df.to_pickle(f'./{timestamp_sec}.test_baa.pickle')
df.to_csv(f'./{timestamp_sec}.test_baa.csv')